In [5]:
from os import system
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False    
if IN_COLAB:
    print('NOTEBOOK RUNNING ON COLAB')
    system('wget https://github.com/patrickhaddadteaching/onetimepad/raw/main/onetimepad_src.py')

NOTEBOOK RUNNING ON COLAB


In [6]:
from onetimepad_src import *
display(v_box_top)